### Environment Setup

In [ ]:
#@title
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title
!git clone https://github.com/abhishtagatya/paintgan
%cd paintgan
!git pull
!pip install tensorflow_addons

In [3]:
#@title
import glob
from PIL import Image


def make_gif(model_name, epoch):
  fpath = "{MODEL_NAME}/results/{MODEL_NAME}_{EPOCH}.png"
  fout = f"{model_name}_{epoch}.gif"

  imgs = []
  for i in range(epoch):
    imgs.append(Image.open(fpath.format(MODEL_NAME=model_name, EPOCH=i+1)))

  imgs = iter(imgs)
  img = next(imgs)
  img.save(fp=fout, format='GIF', append_images=imgs,
          save_all=True, duration=200, loop=0)

### Download Datasets

In [4]:
#@title Kaggle Auth

os.environ['KAGGLE_USERNAME'] = "abhishtagatya" #@param {type:"string"}
os.environ['KAGGLE_KEY'] = "f1266f50ab676ac6cecf736bc495df56" #@param {type:"string"}

In [ ]:
#@title Download Dataset (Alternative)

!unzip /content/drive/MyDrive/Dataset/paintgan-80k_intact.zip

In [ ]:
#@title Download Evaluation Dataset

!python -m scripts.download --source=DRIVE --data-dir=. --dataset=EVAL
!python -m scripts.download --source=DRIVE --data-dir=paintgan-dataset/eval_set.hdf5 --dataset=EVAL_SET
!python -m scripts.download --source=DRIVE --data-dir=paintgan-dataset/eval_model.h5 --dataset=EVAL_MODEL

In [7]:
#@title Download Dataset Config (Kaggle)

os.environ['K_DATA_DIR'] = "./cartoon-dataset" #@param {type:"string"}
os.environ['K_DATASET'] = "volkandl/cartoon-classification" #@param {type:"string"}

In [8]:
#@title Download Dataset (Kaggle)

!python -m scripts.download --source=KAGGLE --data-dir=$K_DATA_DIR --dataset=$K_DATASET

In [9]:
#@title Domain Config

CONTENT_PATH = "./paintgan-dataset/places365/" #@param {type:"string"}
STYLE_PATH = "./paintgan-dataset/wikiart/" #@param {type:"string"}
EVAL_PATH = "./paintgan-eval/" #@param {type:"string"}

DECEPTION_MODEL = "deception_model.h5"
EVAL_DATASET = "eval_set.hdf5"

CONTENT_TEST = "./paintgan-eval/000000000139.jpg" #@param {type:"string"}
STYLE_TEST = "./paintgan-dataset/wikiart/Expressionism/salvador-dali_two-adolescents.jpg" #@param {type:"string"}
DOMAIN_TEST = "vincent-van-gogh" #@param {type:"string"}

FILE_SUFFIX = DOMAIN_TEST + '.jpg'

os.environ['CONTENT_PATH'] = CONTENT_PATH
os.environ['STYLE_PATH'] = STYLE_PATH
os.environ['EVAL_PATH'] = EVAL_PATH

os.environ['DECEPTION_MODEL'] = DECEPTION_MODEL
os.environ['EVAL_DATASET'] = EVAL_DATASET

os.environ['CONTENT_TEST'] = CONTENT_TEST
os.environ['STYLE_TEST'] = STYLE_TEST
os.environ['DOMAIN_TEST'] = DOMAIN_TEST

os.environ['FILE_SUFFIX'] = FILE_SUFFIX

#@markdown ---
#@markdown ### Alternative Domain Config

os.environ['STYLE_PATH_ALT'] = "./cartoon-dataset/cartoon_classification/TRAIN/" #@param {type:"string"}

os.environ['CONTENT_TEST_ALT'] = "./paintgan-eval/Places365_val_00008472.jpg" #@param {type:"string"}
os.environ['STYLE_TEST_ALT'] = "./cartoon-dataset/cartoon_classification/TRAIN/Tsubasa/TSUBASA7370.jpg" #@param {type:"string"}
os.environ['DOMAIN_TEST_ALT'] = "adventure_time" #@param {type:"string"}

os.environ['FILE_SUFFIX_ALT'] = os.environ['DOMAIN_TEST_ALT'] + '.jpg'


### Style Images (Arbitrary)

In [ ]:
style_images_select = {
    'claude-monet': 'paintgan-dataset/wikiart/Impressionism/claude-monet_the-seine-near-giverny-1.jpg',
    'nicholas-roerich': 'paintgan-dataset/wikiart/Symbolism/nicholas-roerich_himalayas-chain-of-kanchenjunga-1936.jpg',
    'pablo-picasso': 'paintgan-dataset/wikiart/Cubism/pablo-picasso_flowers-on-the-table-1907.jpg',
    'paul-cezanne': 'paintgan-dataset/wikiart/Post_Impressionism/paul-cezanne_house-with-red-roof-1890.jpg',
    'salvador-dali': 'paintgan-dataset/wikiart/Expressionism/salvador-dali_two-adolescents.jpg',
    'vincent-van-gogh': 'paintgan-dataset/wikiart/Post_Impressionism/vincent-van-gogh_the-gully-peiroulets-1889.jpg',
}

### A Neural Style Algorithm

In [10]:
#@title Training Config

GATYS_CONTENT_TEST = "./paintgan-eval/000000017031.jpg" #@param {type:"string"}
GATYS_STYLE_TEST = "./paintgan-dataset/wikiart/Post_Impressionism/paul-cezanne_house-with-red-roof-1890.jpg" #@param {type:"string"}
GATYS_EPOCH = "10" #@param {type:"string"}

os.environ['GATYS_CONTENT_TEST'] = GATYS_CONTENT_TEST
os.environ['GATYS_STYLE_TEST'] = GATYS_STYLE_TEST
os.environ['GATYS_EPOCH'] = GATYS_EPOCH

In [ ]:
#@title Training A Neural Style Algorithm

!python -m scripts.train --model=gatys \
  --content=$GATYS_CONTENT_TEST \
  --style=$GATYS_STYLE_TEST \
  --epochs=$GATYS_EPOCH

In [12]:
make_gif('Gatys', int(GATYS_EPOCH))

In [ ]:
#@title Testing A Neural Style Algorithm

!time python -m scripts.test --model=gatys \
  --content=$CONTENT_TEST_ALT \
  --style=$STYLE_TEST_ALT \
  --save-file=$FILE_SUFFIX_ALT

In [ ]:
#@title Evaluating A Neural Style Algorithm

!python -m scripts.evaluate --model=gatys \
  --content=$EVAL_PATH \
  --style=$STYLE_TEST \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Scoring A Neural Style Algorithm

!python -m scripts.score --metric=deception \
  --model=gatys \
  --eval-dir=./Gatys/evaluates/ \
  --eval-dataset=$EVAL_DATASET \
  --checkpoint=$DECEPTION_MODEL

### AdaIN

In [ ]:
#@title Training Config

ADAIN_EPOCH = "50" #@param {type:"string"}
ADAIN_BATCH = "32" #@param {type:"string"}
ADAIN_STEPS = "100" #@param {type:"string"}

ADAIN_DRIVE_CHECKPOINT = "/content/drive/MyDrive/Dataset/adain_50.zip" #@param {type:"string"}
ADAIN_CHECKPOINT_PATH = "/content/paintgan/content/paintgan/AdaIN/checkpoints/AdaIN_50.ckpt" #@param {type:"string"}

os.environ['ADAIN_EPOCH'] = ADAIN_EPOCH
os.environ['ADAIN_BATCH'] = ADAIN_BATCH
os.environ['ADAIN_STEPS'] = ADAIN_STEPS

os.environ['ADAIN_DRIVE_CHECKPOINT'] = ADAIN_DRIVE_CHECKPOINT
os.environ['ADAIN_CHECKPOINT_PATH'] = ADAIN_CHECKPOINT_PATH

In [ ]:
#@title Training AdaIN

!python -m scripts.train --model=adain \
  --content=$CONTENT_PATH \
  --style=$STYLE_PATH \
  --epochs=$ADAIN_EPOCH \
  --batch-size=$ADAIN_BATCH \
  --steps-per-epoch=$ADAIN_STEPS

In [ ]:
make_gif('AdaIN', int(ADAIN_EPOCH))

In [ ]:
#@title Saving Checkpoint

!zip -r adain_50_cartoon.zip /content/paintgan/AdaIN/checkpoints/* 
!mv adain_50_cartoon.zip /content/drive/MyDrive/Dataset/adain_50_cartoon.zip

In [ ]:
#@title Unloading Checkpoint

!unzip $ADAIN_DRIVE_CHECKPOINT

In [ ]:
#@title Testing AdaIN

!time python -m scripts.test --model=adain \
  --content=$CONTENT_TEST  \
  --style=$STYLE_TEST \
  --checkpoint=$ADAIN_CHECKPOINT_PATH \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Evaluating AdaIN

!python -m scripts.evaluate --model=adain \
  --content=$EVAL_PATH \
  --style=$STYLE_TEST \
  --checkpoint=@ADAIN_CHECKPOINT_PATH \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Scoring AdaIN

!python -m scripts.score --metric=deception \
  --model=adain-pc \
  --eval-dir=./AdaIN/evaluates/ \
  --eval-dataset=$EVAL_DATASET \
  --checkpoint=$DECEPTION_MODEL

### CycleGAN

In [ ]:
#@title Training Config

CYCLEGAN_EPOCH = "90" #@param {type:"string"}
CYCLEGAN_BUFFER = "256" #@param {type:"string"}
CYCLEGAN_BATCH = "1" #@param {type:"string"}

CYCLEGAN_DRIVE_CHECKPOINT = "/content/drive/MyDrive/Dataset/cyclegan-pc_90.zip" #@param {type:"string"}
CYCLEGAN_CHECKPOINT_PATH = "/content/paintgan/content/paintgan/CycleGAN/checkpoints/paul-cezanne_90/" #@param {type:"string"}

os.environ['CYCLEGAN_EPOCH'] = CYCLEGAN_EPOCH
os.environ['CYCLEGAN_BUFFER'] = CYCLEGAN_BUFFER
os.environ['CYCLEGAN_BATCH'] = CYCLEGAN_BATCH

os.environ['CYCLEGAN_DRIVE_CHECKPOINT'] = CYCLEGAN_DRIVE_CHECKPOINT
os.environ['CYCLEGAN_CHECKPOINT_PATH'] = CYCLEGAN_CHECKPOINT_PATH

In [ ]:
#@title Training CycleGAN

!python -m scripts.train --model=cyclegan \
  --content=$CONTENT_PATH \
  --style=$STYLE_PATH \
  --style-domain=$DOMAIN_TEST \
  --epochs=$CYCLEGAN_EPOCH \
  --buffer-size=$CYCLEGAN_BUFFER \
  --batch-size=$CYCLEGAN_BATCH

In [ ]:
make_gif('CycleGAN', int(CYCLEGAN_EPOCH))

In [ ]:
#@title Saving Checkpoint

!zip -r ${DOMAIN_TEST}_${CYCLEGAN_EPOCH}.zip /content/paintgan/CycleGAN/checkpoints/${DOMAIN_TEST}_${CYCLEGAN_EPOCH}/* 
!mv ${DOMAIN_TEST}_${CYCLEGAN_EPOCH}.zip /content/drive/MyDrive/Dataset/cyclegan-${DOMAIN_TEST}_${CYCLEGAN_EPOCH}.zip

In [ ]:
#@title Unloading Checkpoint

!unzip $CYCLEGAN_DRIVE_CHECKPOINT

In [ ]:
#@title Testing CycleGAN

!time python -m scripts.test --model=cyclegan \
  --content=$CONTENT_TEST \
  --checkpoint=$CYCLEGAN_CHECKPOINT_PATH \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Evaluating CycleGAN

!python -m scripts.evaluate --model=cyclegan \
  --content=$EVAL_PATH \
  --checkpoint=$CYCLEGAN_CHECKPOINT_PATH \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Scoring CycleGAN

!python -m scripts.score --metric=deception \
  --model=cyclegan-pc \
  --eval-dir=./CycleGAN/evaluates/ \
  --eval-dataset=$EVAL_DATASET \
  --checkpoint=$DECEPTION_MODEL

### DiscoGAN

In [18]:
#@title Training Config

DISCOGAN_EPOCH = "20" #@param {type:"string"}
DISCOGAN_BATCH = "1" #@param {type:"string"}

DISCOGAN_DRIVE_CHECKPOINT = "/content/drive/MyDrive/Dataset/pgan-pc_1_20.zip" #@param {type:"string"}
DISCOGAN_CHECKPOINT_PATH = "/content/paintgan/content/paintgan/PaintGAN/checkpoints/paul-cezanne_90/" #@param {type:"string"}

os.environ['DISCOGAN_EPOCH'] = DISCOGAN_EPOCH
os.environ['DISCOGAN_BATCH'] = DISCOGAN_BATCH

os.environ['DISCOGAN_DRIVE_CHECKPOINT'] = DISCOGAN_DRIVE_CHECKPOINT
os.environ['DISCOGAN_CHECKPOINT_PATH'] = DISCOGAN_CHECKPOINT_PATH

In [ ]:
#@title Training PaintGAN

!python -m scripts.train --model=discogan \
  --content=$CONTENT_PATH \
  --style=$STYLE_PATH \
  --style-domain=$DOMAIN_TEST \
  --epochs=$DISCOGAN_EPOCH \
  --batch-size=$DISCOGAN_BATCH

In [ ]:
make_gif('DiscoGAN', int(DISCOGAN_EPOCH))

In [ ]:
#@title Saving Checkpoint

!zip -r ${DOMAIN_TEST}_${DISCOGAN_EPOCH}.zip /content/paintgan/DiscoGAN/checkpoints/${DOMAIN_TEST}_${DISCOGAN_EPOCH}/* 
!mv ${DOMAIN_TEST}_${DISCOGAN_EPOCH}.zip /content/drive/MyDrive/Dataset/paintgan-${DOMAIN_TEST}_${DISCOGAN_EPOCH}.zip

In [ ]:
#@title Unloading Checkpoint

!unzip $DISCOGAN_DRIVE_CHECKPOINT

In [ ]:
#@title Testing PaintGAN

!time python -m scripts.test --model=discogan \
  --content=$CONTENT_TEST \
  --checkpoint=$DISCOGAN_CHECKPOINT_PATH \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Evaluating PaintGAN

!python -m scripts.evaluate --model=discogan \
  --content=$CONTENT_TEST \
  --checkpoint=$DISCOGAN_CHECKPOINT_PATH \
  --save-file=$FILE_SUFFIX

In [ ]:
#@title Scoring PaintGAN

!python -m scripts.score --metric=deception \
  --model=discogan-sd \
  --eval-dir=./DiscoGAN/evaluates/ \
  --eval-dataset=$EVAL_DATASET \
  --checkpoint=$DECEPTION_MODEL